## imports, etc.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
import os
import sys
import pickle as pkl
import numpy as np
from sklearn.semi_supervised import LabelPropagation
from sklearn.neural_network import MLPClassifier

In [2]:
np.random.seed(0)

In [3]:
X_train = torch.load("data/FashionMNIST_train_latent.pt")
X_test = torch.load("data/FashionMNIST_test_latent.pt")
y_train = torch.load("data/FashionMNIST_train_labels.pt")
y_test = torch.load("data/FashionMNIST_test_labels.pt")

In [4]:
os.chdir("../fashion_ml")

In [5]:
from fashion_vae import FashionVAE
from data_loader import EZ_Dataloader
from experiment import FashionML

In [19]:
import warnings
warnings.filterwarnings("ignore")

## limit data labels

In [12]:
X = np.array(torch.cat((X_train, X_test), 0))
y = np.array(torch.cat((y_train, y_test), 0))

discard all but 75 random labels (saw very poor performance with <=60 labels)

In [13]:
ind = np.random.randint(len(X), size=(75))
y_lim = np.repeat(-1, len(y))
y_lim[ind] = y[ind]

## semi-supervised clustering

search for optimal k: 2 is optimal, anything over 3 predicts everything as the first class

In [18]:
def test_knn(n):
    lpm = LabelPropagation(kernel='knn',n_neighbors=n,n_jobs=-1,max_iter=3000)
    lpm.fit(X, y_lim)
    print(lpm.score(X, y))
    y_pred = lpm.predict(X)
    print(np.unique(y_pred))
    return lpm

In [20]:
test_knn(2)

0.10095714285714286
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1, n_neighbors=2)

In [21]:
test_knn(3)

0.10064285714285715
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1, n_neighbors=3)

In [22]:
test_knn(7)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1)

In [23]:
test_knn(12)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1, n_neighbors=12)

In [24]:
test_knn(35)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=3000, n_jobs=-1, n_neighbors=35)

final clusters:

In [25]:
lpm = LabelPropagation(kernel='knn',n_neighbors=2,n_jobs=-1,max_iter=3500)
lpm.fit(X, y_lim)
y_pred = lpm.predict(X)

## mlp models

pull out train vs. test:

In [30]:
X_train = X[y_lim != -1]
X_test = X[y_lim == -1]
y_train = y_lim[y_lim != -1]
y_test = y_lim[y_lim == -1]

In [41]:
len(X_train)

75

In [42]:
len(X_test)

69925

test a few networks:

In [55]:
clf = MLPClassifier(hidden_layer_sizes=(100,20,5), tol=1e-5)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 20, 5), tol=1e-05)

In [56]:
clf.score(X_test, y_test)

0.0

In [57]:
clf.predict(X_test)

array([9, 4, 7, ..., 2, 1, 7])

In [58]:
clf = MLPClassifier(hidden_layer_sizes=(20,5), tol=1e-5)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(20, 5), tol=1e-05)

In [59]:
clf.score(X_test, y_test)

0.0

In [60]:
clf.predict(X_test)

array([2, 2, 2, ..., 2, 1, 2])

In [61]:
clf = MLPClassifier(hidden_layer_sizes=(10), tol=1e-5)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=10, tol=1e-05)

In [62]:
clf.score(X_test, y_test)

0.0

In [63]:
clf.predict(X_test)

array([8, 0, 1, ..., 0, 1, 1])

not looking very good lol